In [7]:
import requests
import pandas as pd
import os

os.makedirs("data/raw", exist_ok=True)

In [8]:
def get_price_history(coin_id, vs_currency="usd", days="365", interval="daily"):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {
        "vs_currency": vs_currency,
        "days": days,
        "interval": interval
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data for {coin_id}")
        return None
    data = response.json()
    prices = data["prices"]
    df = pd.DataFrame(prices, columns=["timestamp", f"{coin_id}_price"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms").dt.date
    df = df.drop(columns=["timestamp"])
    return df

In [9]:
fxs_df = get_price_history("frax-share")
frax_df = get_price_history("frax")

price_df = pd.merge(fxs_df, frax_df, on="date")
price_df.head()

,frax-share_price,date,frax_price
0,1.803753,2024-08-07,0.996601
1,1.729273,2024-08-08,0.996023
2,1.918425,2024-08-09,0.995463
3,1.918014,2024-08-10,0.999430
4,1.920749,2024-08-11,0.996680


In [10]:
price_df = pd.merge(fxs_df, frax_df, on="date")
price_df.to_csv("data/raw/fxs_frax_price_history.csv", index=False)
print("Saved.")


Saved.


In [13]:
def get_frax_tvl():
    url = "https://api.llama.fi/protocol/frax"
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to fetch TVL data.")
        return None
    data = response.json()
    tvl_data = data["tvl"]
    df = pd.DataFrame(tvl_data)
    df["date"] = pd.to_datetime(df["date"], unit="s").dt.date
    df = df.rename(columns={"totalLiquidityUSD": "tvl_usd"})
    return df[["date", "tvl_usd"]]

tvl_df = get_frax_tvl()
if tvl_df is not None:
    tvl_df.to_csv("data/raw/frax_tvl.csv", index=False)
    print("TVL data saved.")
else:
    print("TVL data was not saved.")

TVL data saved.
